# Example use-case: cross-match ZTF BTS and NGC

Here we demonstrate how to cross-match [Zwicky Transient Facility](https://ztf.caltech.edu) (ZTF) [Bright Transient Survey](https://sites.astro.caltech.edu/ztf/bts) (BTS) and [New General Catalogue](https://en.wikipedia.org/wiki/New_General_Catalogue) (NGC) using LSDB.

In [ ]:
# Install astroquery, comment this line if you already have it
!pip install --quiet astroquery

In [ ]:
import lsdb
import pandas as pd
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier
from dask.distributed import Client

### Download ZTF BTS and convert coordinates to degrees

In [ ]:
%%time

df_ztf_bts = pd.read_csv(
    'http://sites.astro.caltech.edu/ztf/bts/explorer.php?format=csv',
    na_values='-',
)
coord = SkyCoord(df_ztf_bts['RA'], df_ztf_bts['Dec'], unit=('hourangle', 'deg'))
df_ztf_bts['ra_deg'], df_ztf_bts['dec_deg'] = coord.ra.deg, coord.dec.deg
df_ztf_bts.head()

### Download NGC with `astroquery`

Please install astroquery first with `pip install astroquery` or `conda install -c conda-forge astroquery`.

In [ ]:
%%time

vizier = Vizier(row_limit=50_000)
tables = vizier.get_catalogs('VII/118/ngc2000')
df_ngc = tables[0].to_pandas()
coord = SkyCoord(df_ngc['RAB2000'], df_ngc['DEB2000'], unit=('hourangle', 'deg'))
df_ngc['ra_deg'], df_ngc['dec_deg'] = coord.ra.deg, coord.dec.deg
df_ngc.head()

### Put both catalogs to LSDB and plan cross-match

Of course ZTF looks much deeper than NGC galaxies from 19th century, so we filter ZTF transients by redshift.

LSDB is built upon [Dask](https://dask.org) and can be used with Dask distributed cluster. In this cell we just plan computations and do not actually run them.

In [ ]:
%%time

ztf_bts = lsdb.from_dataframe(df_ztf_bts, ra_column='ra_deg', dec_column='dec_deg')
ngc = lsdb.from_dataframe(df_ngc, ra_column='ra_deg', dec_column='dec_deg')

ztf_bts = ztf_bts.query('redshift < 0.01')

matched = ztf_bts.crossmatch(ngc, radius_arcsec=1200, suffixes=('_ztf', '_ngc'))
matched

### Run LSDB pipeline

In [ ]:
%%time

# Create default local cluster
with Client():
    matched_df = matched.compute()

# Let's output transient name, NGC name and angular distance between them
matched_df[['IAUID_ztf', 'Name_ngc', '_DIST']]

We may have some false matches here, because NGC is too shallow for this task. However, if we look to the second and third row we would see that [AT2019rsi](https://www.wis-tns.org/object/2019rsi) and [AT2019sxc](https://www.wis-tns.org/object/2019sxc) are Novae in M31.